# [Version en français ici!](./Get_Projects_Tiles_by_AOI_FR.ipynb)
# Identify projects and lidar tiles covering a region of interest

This tutorial gives examples of how to find lidar projects and tiles covering a region of interest. We use the index files of the product [LiDAR Point Clouds - CanElevation Series](https://open.canada.ca/data/en/dataset/7069387e-9986-4297-9f55-0288e9676947). Primarily, we use the [geopandas](https://geopandas.org/en/stable/) module to manipulate the index layers and perform spatial queries.
In a real application, the list of tiles obtained for a region of interest could be used to build a more advanced processing workflow as described in the notebook [creating a digital elevation model (DEM) from a lidar point cloud in COPC LAZ format](./DEM_from_COPC_lidar_EN.ipynb).


## Steps:
* [Preparing the environment](#Preparing-the-environment)
* [Importing the necessary modules](#Importing-the-necessary-modules)
* [Download product indexes](#Download-product-indexes)
* [Creating geopandas objects](#Creating-geopandas-objects)
* [Find projects and tiles covering an area of interest](#Find-projects-and-tiles-covering-an-area-of-interest)
* [Using a region of interest in another projection](#Using-a-region-of-interest-in-another-projection)

## Preparing the environment
Three steps are required to run these examples locally.
### 1. retrieve the source code
   In the location of your choice on your computer. Retrieve the source code of the examples by running a git clone command.

   >```bash
   >git clone https://github.com/NRCan/CanElevation.git
   >```

   Then go to the directory containing the notebooks.

   >```bash
   >cd CanElevation/pointclouds_nuagespoints
   >```


### 2. conda installation

   We recommend using conda to install the dependencies required for proper operation. Detailed [installation instructions](https://docs.anaconda.com/miniconda/install/#quick-command-line-install) are available. The following instructions assume that conda is available from the command line.

### 3. Installing dependencies

   The requirements.yaml file contains the necessary dependencies. We therefore need to create a conda environment with these dependencies.

   >```bash
   >conda env create -n CanElevation_PointClouds --file requirements.yml
   >```  

   Next, we need to activate this newly created conda environment. 
	
   >```bash
   >conda activate CanElevation_PointClouds
   >```
    
To launch jupyter notebook, simply use the following command:

   >````bash
   >jupyter notebook
   >```
    
In case of problems or for more launch options, please refer to the [user guide](https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/execute.html).

**You can now launch the code for the other cells in the notebook**.

## Importing the necessary modules
In the following examples, we mainly use the geopandas library. We also use the requests module to download indexes and the zipfile module to decompress them locally.

In [3]:
import os
import requests
import zipfile
import geopandas as gpd
from shapely import box
from IPython.display import display
from ipywidgets import Output

## Download product indexes
The [product page](https://open.canada.ca/data/en/dataset/7069387e-9986-4297-9f55-0288e9676947) contains the indexes of the distributed lidar projects and the lidar tiles making up each project. Before we can manipulate them, we need to download them and extract the contents of the compressed files.


In [4]:
# URLs to the shapefiles containing LiDAR projects and tiles index
# Liens URLs des index de projets et tuiles lidar
projects_url = 'https://download-telecharger.services.geo.ca/pub/elevation/pointclouds_nuagespoints/Index_LiDARprojects_projetslidar.zip'
tiles_url = 'https://download-telecharger.services.geo.ca/pub/elevation/pointclouds_nuagespoints/Index_LiDARtiles_tuileslidar.zip'

for index_url in (projects_url, tiles_url):
    zipname = os.path.basename(index_url)
    basename = os.path.splitext(zipname)[0]
    
    # Downloading / téléchargement
    response = requests.get(index_url)
    with open(zipname, 'wb') as file:
        file.write(response.content)

    #Unzipping / extraction
    with zipfile.ZipFile(zipname, 'r') as zip_ref:
        zip_ref.extractall(basename)
    

## Creating geopandas objects
We'll be using the Geopandas library to interact with the indexes. This library supports both spatial and geometric operations. Please consult the library documentation for more information.

In [5]:
# Loading the index in geopandas objects. The coordinate reference system of the index is NAD83(CSRS) - EPSG:4617
# Chargement des index dans des objets geopandas. Le système de référence de ces index est le NAD83(CSRS) - EPSG:4617
gdf_projects = gpd.read_file('Index_LiDARprojects_projetslidar/Index_LiDARprojects_projetslidar.shp')

# Load the lidar tiles / Chargement des tuiles lidars
gdf_tiles = gpd.read_file('Index_LiDARtiles_tuileslidar/Index_LiDARtiles_tuileslidar.shp')

/space/partner/nrcan/geobase/work/opt/miniconda-elevation/envs/lidar-demo/lib/python3.13/site-packages/pyogrio/raw.py:198: RuntimeWarning: Index_LiDARtiles_tuileslidar/Index_LiDARtiles_tuileslidar.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


## Find projects and tiles covering an area of interest
A common use case is to determine the lidar projects and tiles covering a region of interest. In this example, we define an area of interest. We then determine the projects and tiles covering this area using a spatial intersection query. The results are printed on screen.

In [6]:

aoi = box(-80, 43.20, -79.971, 43.225)

aoi_gdf = gpd.GeoDataFrame({'geometry': [aoi]}, crs=4617)

overlapping_projects = gdf_projects[gdf_projects.intersects(aoi_gdf.geometry.iloc[0])]
overlapping_tiles = gdf_tiles[gdf_tiles.intersects(aoi_gdf.geometry.iloc[0])]

out = Output()
with out:
    print(f"Overlapping LiDAR Projects:")
    for _, row in overlapping_projects.iterrows():
        print(f"Project: {row['Project']}, URL: {row['URL']}")

    print(f"Overlapping LiDAR Tiles:")
    for _, row in overlapping_tiles.iterrows():
        print(f"Project: {row['Project']}, Tile name: {row['Tile_name']}")
display(out)

Output()

## Using a region of interest in another projection
Are you working with projected coordinates? If you know the projection code, it's easy to reproject them before performing the intersection operation on projects and tiles. 
Here we adapt the previous example to find the projects and tiles covering the projected area of interest.

In [7]:
# AOI coordinates / Coordonnées de la région d'intérêt - NAD83(CSRS) / UTM zone 17N
aoi_utm = box(580736, 4783785, 583114, 4785855)

# We specify the UTM 17N EPSG code / Nous utilisons ici le code EPSG UTM zone 17N
aoi_gdf = gpd.GeoDataFrame({'geometry': [aoi_utm]}, crs=2958)

# The extra step needed is the reprojection of the AOI in 4617
# L'étape additionnelle requise est la reprojection de la zone d'intérêt en 4617
aoi_gdf = aoi_gdf.to_crs(epsg=4617)


# The remaining steps are the same / Les autres opérations sont inchangées
overlapping_projects = gdf_projects[gdf_projects.intersects(aoi_gdf.geometry.iloc[0])]
overlapping_tiles = gdf_tiles[gdf_tiles.intersects(aoi_gdf.geometry.iloc[0])]

out = Output()
with out:
    print(f"Overlapping LiDAR Projects:")
    for _, row in overlapping_projects.iterrows():
        print(f"Project: {row['Project']}, URL: {row['URL']}")

    print(f"Overlapping LiDAR Tiles:")
    for _, row in overlapping_tiles.iterrows():
        print(f"Project: {row['Project']}, Tile name: {row['Tile_name']}")
display(out)

Output()